In [7]:
import os, re, sys

# stage_in packages
from unity_sds_client.resources.collection import Collection

# stage_out packages
from datetime import datetime, timezone
from unity_sds_client.resources.dataset import Dataset
from unity_sds_client.resources.data_file import DataFile

In [8]:
input_stac_collection_file = 'test/stage_in/catalog.json' # type: stage-in
output_stac_catalog_dir    = 'process_results/'                    # type: stage-out

In [9]:
inp_collection = Collection.from_stac(input_stac_collection_file)
data_filenames = inp_collection.data_locations()

data_filenames

if2


['/unity/ads/input_collections/EMIT_TEST/EMIT_L1B/EMIT_L1B_RAD_001_20230620T084426_2317106_011.nc',
 '/unity/ads/input_collections/EMIT_TEST/EMIT_L1B/EMIT_L1B_OBS_001_20230620T084426_2317106_011.nc',
 '/unity/ads/input_collections/EMIT_TEST/EMIT_L2A/EMIT_L2A_RFL_001_20230620T084426_2317106_011.nc',
 '/unity/ads/input_collections/EMIT_TEST/EMIT_L2A/EMIT_L2A_RFLUNCERT_001_20230620T084426_2317106_011.nc',
 '/unity/ads/input_collections/EMIT_TEST/EMIT_L2A/EMIT_L2A_MASK_001_20230620T084426_2317106_011.nc']

In [10]:
r = re.compile('.*L1B_RAD*')
l1b_rad_filename = list(filter(r.match, data_filenames))[0]
r = re.compile('.*L1B_OBS*')
l1b_obs_filename = list(filter(r.match, data_filenames))[0]
r = re.compile('.*L2A_MASL*')
l2a_mask_filename = list(filter(r.match, data_filenames))[0]

#print(l1b_rad_filename)
#print(l1b_obs_filename)
#print(l2a_mask_filename)

In [12]:
sys.argv = [l1b_rad_filename, l1b_obs_filename, l1b_rad_filename, 'blah.img', l2a_mask_filename, l2a_mask_filename, './process_results/test_output', '--state_subs ', l2a_mask_filename]
exec(open("./ghg_process.py").read())

ImportError: cannot import name 'ReadAbstractDataSet' from 'utils' (/unity/ads/users/gshen/jay_emit_testing/emit-ghg/utils.py)

In [13]:
print(sys.argv)

['/unity/ads/input_collections/EMIT_TEST/EMIT_L1B/EMIT_L1B_RAD_001_20230620T084426_2317106_011.nc', '/unity/ads/input_collections/EMIT_TEST/EMIT_L1B/EMIT_L1B_OBS_001_20230620T084426_2317106_011.nc', '/unity/ads/input_collections/EMIT_TEST/EMIT_L1B/EMIT_L1B_RAD_001_20230620T084426_2317106_011.nc', 'blah.img', '/unity/ads/input_collections/EMIT_TEST/EMIT_L2A/EMIT_L2A_MASK_001_20230620T084426_2317106_011.nc', '/unity/ads/input_collections/EMIT_TEST/EMIT_L2A/EMIT_L2A_MASK_001_20230620T084426_2317106_011.nc', './process_results/test_output', '--state_subs ', '/unity/ads/input_collections/EMIT_TEST/EMIT_L2A/EMIT_L2A_MASK_001_20230620T084426_2317106_011.nc']
